In [2]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import os

### Loading Price Data

In [3]:
project_root = os.path.abspath(os.getcwd())

data_dir = os.path.join(project_root, '..', '..', 'data')

# Summer month

p_summer = pd.read_csv(os.path.join(data_dir, 'Spotmarket_August_2024.csv'),sep=';')

# Winter month

p_winter = pd.read_csv(os.path.join(data_dir, 'Spotmarket_December_2024.csv'),sep=';')

#Spotmarket data from: https://energy-charts.info/charts/price_spot_market/chart.htm?l=en&c=CH&interval=month&year=2024&legendItems=by4&month=12


### Loading Fixed Appliances Data

In [4]:
#TV consumption data for summer month
tv_summer = pd.read_csv(os.path.join(data_dir, 'tv_consumption_august_2024_detailed.csv'), sep=',')

#TV consumption data for winter month
tv_winter = pd.read_csv(os.path.join(data_dir, 'tv_consumption_december_2024_detailed.csv'), sep=',')

#Lighting consumption data for summer month
lighting_summer = pd.read_csv(os.path.join(data_dir, 'lighting_consumption_august_2024.csv'), sep=',')

#Lighting consumption data for winter month
lighting_winter = pd.read_csv(os.path.join(data_dir, 'lighting_consumption_december_2024.csv'), sep=',')

#Fridge consumption data for summer month
fridge_summer = pd.read_csv(os.path.join(data_dir, 'fridge_August_2024.csv'), sep=';')

#Fridge consumption data for winter month
fridge_winter = pd.read_csv(os.path.join(data_dir, 'fridge_December_2024.csv'), sep=';')

#Oven consumption data for summer month
oven_summer = pd.read_csv(os.path.join(data_dir, 'Oven_Energy_Consumption_August_2024.csv'),sep=';')

#Oven consumption data for winter month
oven_winter = pd.read_csv(os.path.join(data_dir, 'Oven_Energy_Consumption_December_2024.csv'),sep=';')

#Induction stove consumption data for summer month
induction_summer = pd.read_csv(os.path.join(data_dir, 'Induction_Stove_Energy_Consumption_August_2024.csv'),sep=';')

#Induction stove consumption data for winter month
induction_winter = pd.read_csv(os.path.join(data_dir, 'Induction_Stove_Energy_Consumption_December_2024.csv'),sep=';')


### Data formatting

In [5]:
#get column names
p_summer.columns = p_summer.columns.str.replace(' ', '_')
p_winter.columns = p_winter.columns.str.replace(' ', '_')
tv_summer.columns = tv_summer.columns.str.replace(' ', '_')
tv_winter.columns = tv_winter.columns.str.replace(' ', '_')
lighting_summer.columns = lighting_summer.columns.str.replace(' ', '_')
lighting_winter.columns = lighting_winter.columns.str.replace(' ', '_')
fridge_summer.columns = fridge_summer.columns.str.replace(' ', '_')
fridge_winter.columns = fridge_winter.columns.str.replace(' ', '_')
oven_summer.columns = oven_summer.columns.str.replace(' ', '_')
oven_winter.columns = oven_winter.columns.str.replace(' ', '_')
induction_summer.columns = induction_summer.columns.str.replace(' ', '_')
induction_winter.columns = induction_winter.columns.str.replace(' ','_')


### Optimization

In [6]:
import pandas as pd
import gurobipy as gp
import itertools
from gurobipy import GRB

# Load data
price = p_summer["Price_(EUR/MWh)"].values
fridge = fridge_summer["Energy_kWh_1h"].values
stove = induction_summer["Energy_Consumption_kWh"].values
tv = tv_summer["tv_power_kWh"].values
lighting = lighting_summer["lighting_power_kWh"].values

Time_interval = len(price)

# Dishwasher properties
duration = 3  # hours of operation
min_gap = 15  # hours between runs
power_dishwasher = 1.5  # kW during operation

# Gurobi model
model = gp.Model("automated_demand_response")

# Binary start variables: 1 if dishwasher starts at hour t
start_times = range(Time_interval - duration + 1)
dishwasher_start = model.addVars(start_times, vtype=GRB.BINARY, name="start")

# Binary on variables: 1 if dishwasher is on at hour t
binary_dishwasher = model.addVars(Time_interval, vtype=GRB.BINARY, name="on")

for t, k in itertools.product(range(len(start_times)), range(duration)):
    model.addConstr(binary_dishwasher[t + k] >= dishwasher_start[t], name=f"dishwasher_{t}_{k}")

for t in range(len(start_times)-1):
    model.addConstr(gp.quicksum(binary_dishwasher[t + k] for k in range(duration+1)) <= 3, name=f"max_three_hours_on_{t}")


# Enforce min 1 run per day
#adds up all possible start times of the dishwasher in a single and '>= 1' day ensures that the dishwasher has to run once per day
hours_per_day = 24
days = Time_interval // hours_per_day
for d in range(days-1):
    model.addConstr(gp.quicksum(dishwasher_start[t] for t in range(d * 24, (d + 1) * 24)) == 1,
                name=f"max_one_run_per_day_dishwasher_{d}")
    
# Enforce max 1 run per day of max 3 hours
#adds up all possible start times of the dishwasher in a single and '<= 1' day ensures that the dishwasher doesn't run more than once per day
hours_per_day = 24
days = Time_interval // hours_per_day
for d in range(days-1):
    model.addConstr(gp.quicksum(binary_dishwasher[t] for t in range(d * 24, (d + 1) * 24)) == 3,
                name=f"max_one_run_per_day_dishwasher_{d}")

# Enforce minimum gap (15 hours) between two starts
#multiplying with min_gap so we don't constrain the dishwasher to start at the same time every day
#this enures that the optimizer can iterate through without being constrained to a single time
for t in range(len(start_times)-min_gap):
    model.addConstr(gp.quicksum(dishwasher_start[t + offset] for offset in range(1, min_gap + 1)) <= (1 - dishwasher_start[t]) * min_gap,
                name=f"min_gap_after_{t}")
#print(len(fridge))
#print(len(stove))
#print(len(tv))
#print(len(lighting))
#print(len(price))
#print(len(dishwasher_start))
#print(len(binary_dishwasher))
#print(len(start_times))
# Total power consumption including fixed and dishwasher
total_load = [fridge[t] + stove[t] + tv[t] + lighting[t] + power_dishwasher * binary_dishwasher[t] for t in range(0,720)]

# Objective: minimize total electricity cost
model.setObjective(gp.quicksum(price[t] * total_load[t] for t in range(720)), GRB.MINIMIZE)

# Optimize
model.optimize()

dishwasher_starts_list = []
# Output dishwasher schedule
if model.status == GRB.OPTIMAL:
    print(f"Total cost: {model.ObjVal:.2f}")
    for t in range(Time_interval):
        if dishwasher_start.get(t) and dishwasher_start[t].X > 0.5:
            print(f"Dishwasher starts at: {p_summer['Date_(GMT+2)'][t]}")
            dishwasher_starts_list.append(p_summer['Date_(GMT+2)'][t])


Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3754 rows, 1486 columns and 20488 nonzeros
Model fingerprint: 0x5829b74e
Variable types: 0 continuous, 1486 integer (1486 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 2449 rows and 45 columns
Presolve time: 0.08s
Presolved: 1305 rows, 1441 columns, 11634 nonzeros
Variable types: 0 continuous, 1441 integer (1441 binary)
Found heuristic solution: objective 42366.678172

Root relaxation: objective 3.959426e+04, 260 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node

In [7]:
print("Dishwasher schedule:")
print(dishwasher_starts_list)

Dishwasher schedule:
['01.08.24 13:00', '02.08.24 07:00', '03.08.24 13:00', '04.08.24 13:00', '05.08.24 12:00', '06.08.24 12:00', '07.08.24 13:00', '08.08.24 13:00', '09.08.24 12:00', '10.08.24 12:00', '11.08.24 12:00', '12.08.24 12:00', '13.08.24 12:00', '14.08.24 21:00', '15.08.24 13:00', '16.08.24 12:00', '17.08.24 13:00', '18.08.24 13:00', '19.08.24 12:00', '20.08.24 12:00', '21.08.24 13:00', '22.08.24 12:00', '23.08.24 12:00', '24.08.24 13:00', '25.08.24 13:00', '26.08.24 12:00', '27.08.24 12:00', '28.08.24 12:00', '29.08.24 12:00', '30.08.24 12:00']


In [8]:
#print timestamp and the corresponding state of the dishwasher
for t in range(Time_interval):
    if binary_dishwasher[t].X > 0.5:
        print(f"Dishwasher is ON at: {p_summer['Date_(GMT+2)'][t]}")
    if binary_dishwasher[t].X < 0.5:
        print(f"Dishwasher is OFF at: {p_summer['Date_(GMT+2)'][t]}")


Dishwasher is OFF at: 01.08.24 00:00
Dishwasher is OFF at: 01.08.24 01:00
Dishwasher is OFF at: 01.08.24 02:00
Dishwasher is OFF at: 01.08.24 03:00
Dishwasher is OFF at: 01.08.24 04:00
Dishwasher is OFF at: 01.08.24 05:00
Dishwasher is OFF at: 01.08.24 06:00
Dishwasher is OFF at: 01.08.24 07:00
Dishwasher is OFF at: 01.08.24 08:00
Dishwasher is OFF at: 01.08.24 09:00
Dishwasher is OFF at: 01.08.24 10:00
Dishwasher is OFF at: 01.08.24 11:00
Dishwasher is OFF at: 01.08.24 12:00
Dishwasher is ON at: 01.08.24 13:00
Dishwasher is ON at: 01.08.24 14:00
Dishwasher is ON at: 01.08.24 15:00
Dishwasher is OFF at: 01.08.24 16:00
Dishwasher is OFF at: 01.08.24 17:00
Dishwasher is OFF at: 01.08.24 18:00
Dishwasher is OFF at: 01.08.24 19:00
Dishwasher is OFF at: 01.08.24 20:00
Dishwasher is OFF at: 01.08.24 21:00
Dishwasher is OFF at: 01.08.24 22:00
Dishwasher is OFF at: 01.08.24 23:00
Dishwasher is OFF at: 02.08.24 00:00
Dishwasher is OFF at: 02.08.24 01:00
Dishwasher is OFF at: 02.08.24 02:00
Dish

In [48]:
#print dishwasher start times
for t in range(Time_interval):
    if dishwasher_start[t].X > 0.5:
        print(f"Dishwasher start time: {p_summer['Date_(GMT+2)'][t]}")

Dishwasher start time: 01.08.24 13:00
Dishwasher start time: 02.08.24 07:00
Dishwasher start time: 03.08.24 13:00
Dishwasher start time: 04.08.24 13:00
Dishwasher start time: 05.08.24 12:00
Dishwasher start time: 06.08.24 12:00
Dishwasher start time: 07.08.24 13:00
Dishwasher start time: 08.08.24 13:00
Dishwasher start time: 09.08.24 12:00
Dishwasher start time: 10.08.24 12:00
Dishwasher start time: 11.08.24 13:00
Dishwasher start time: 12.08.24 12:00
Dishwasher start time: 13.08.24 12:00
Dishwasher start time: 14.08.24 21:00
Dishwasher start time: 15.08.24 13:00
Dishwasher start time: 16.08.24 12:00
Dishwasher start time: 17.08.24 13:00
Dishwasher start time: 18.08.24 13:00
Dishwasher start time: 19.08.24 12:00
Dishwasher start time: 20.08.24 12:00
Dishwasher start time: 21.08.24 13:00
Dishwasher start time: 22.08.24 12:00
Dishwasher start time: 23.08.24 12:00
Dishwasher start time: 24.08.24 13:00
Dishwasher start time: 25.08.24 10:00
Dishwasher start time: 26.08.24 12:00
Dishwasher s

KeyError: 742